<a href="https://colab.research.google.com/github/AmitS1009/ML_Algo/blob/main/NLP_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Dependicies :

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


Data Collecting :

In [5]:
data = pd.read_csv('/content/IMDB Dataset.csv', encoding='utf-8')

In [6]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
data.shape

(50000, 2)

Data Preprocessing :

In [8]:
X = data['review']

In [9]:
print(X)

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object


In [10]:
y = data['sentiment']

In [11]:
print(y)

0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
49995    positive
49996    negative
49997    negative
49998    negative
49999    negative
Name: sentiment, Length: 50000, dtype: object


In [12]:
#lowercasing the words :

data.review = data.review.str.lower()

In [13]:
data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [14]:
# removing the Html tags :

import re

def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub('', text)

In [15]:
data.review = data.review.apply(remove_html_tags)

In [16]:
data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [17]:
# removing url :

def remove_url(text):
  pattern = re.compile(r'https?://\S+/\S*')
  return pattern.sub('', text)

In [18]:
data.review = data.review.apply(remove_url)

In [19]:




#remove puntuatio :

def remove_punc(text):
  pattern = re.compile(r'[^a-zA-Z0-9\s]')
  return pattern.sub('', text)

In [20]:
review = data.review.apply(remove_punc)

In [21]:
data.tail()

,review,sentiment
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative
49999,no one expects the star trek movies to be high...,negative


In [22]:
chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek You",
    "ILU": "I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your Sex And Age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That Feeling When",
    "MFW": "My Face When",
    "MRW": "My Reaction When",
    "IFYP": "I Feel Your Pain",
    "TNTL": "Trying Not To Laugh",
    "JK": "Just Kidding",
    "IDC": "I Don’t Care",
    "ILY": "I Love You",
    "IMU": "I Miss You",
    "ADIH": "Another Day In Hell",
    "ZZZ": "Sleeping, Bored, Tired",
    "WYWH": "Wish You Were Here",
    "TIME": "Tears In My Eyes",
    "BAE": "Before Anyone Else",
    "FIMH": "Forever In My Heart",
    "BSAAW": "Big Smile And A Wink",
    "BWL": "Bursting With Laughter",
    "BFF": "Best Friends Forever",
    "CSL": "Can’t Stop Laughing"
}


In [23]:
#chat word treatment :

def chat_convo(text):
  new_text = []

  for w in text.split():
    if w.upper() in chat_words :
      new_text.append(chat_words[w.upper()])
    else :
      new_text.append(w)
  return " ".join(new_text)

In [24]:
review = data.review.apply(chat_convo)

In [25]:
review.head()

,review
0,one of the other reviewers has mentioned that ...
1,a wonderful little production. the filming tec...
2,i thought this was a wonderful way to spend Te...
3,basically there's a family where a little boy ...
4,"petter mattei's ""love in the Tears In My Eyes ..."


In [26]:
!pip install pyspellchecker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.6 MB/s eta 0:00:00


In [27]:
#spelling correction : we'll do it letter :

"""from textblob import TextBlob      # slow, so not using it

def correct_spelling(text):
    corrected_text = str(TextBlob(text).correct())
  return corrected_text


from spellchecker import SpellChecker
import pandas as pd

# Initialize SpellChecker
spell = SpellChecker()

# Define the optimized spelling correction function
def correct_spelling_optimized(text):
    if pd.isna(text):  # Handle NaN values
        return text
    corrected_words = [spell.correction(word) if spell.correction(word) else word for word in text.split()]
    return " ".join(corrected_words)

# Process the data in chunks to avoid memory issues
chunk_size = 500  # Adjust chunk size as needed
for start in range(0, len(data), chunk_size):
    end = start + chunk_size
    data.loc[start:end, 'review'] = data.loc[start:end, 'review'].apply(correct_spelling_optimized)"""


'from textblob import TextBlob      # slow, so not using it\n\ndef correct_spelling(text):\n    corrected_text = str(TextBlob(text).correct())\n  return corrected_text\n\n\nfrom spellchecker import SpellChecker\nimport pandas as pd\n\n# Initialize SpellChecker\nspell = SpellChecker()\n\n# Define the optimized spelling correction function\ndef correct_spelling_optimized(text):\n    if pd.isna(text):  # Handle NaN values\n        return text\n    corrected_words = [spell.correction(word) if spell.correction(word) else word for word in text.split()]\n    return " ".join(corrected_words)\n\n# Process the data in chunks to avoid memory issues\nchunk_size = 500  # Adjust chunk size as needed\nfor start in range(0, len(data), chunk_size):\n    end = start + chunk_size\n    data.loc[start:end, \'review\'] = data.loc[start:end, \'review\'].apply(correct_spelling_optimized)'

In [34]:
#removing stopwrods :

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
from nltk.corpus import stopwords

# Load stopwords once
stop_words = set(stopwords.words('english'))

def remove_stopwords_fast(text_series):
    return text_series.apply(
        lambda text: " ".join([word for word in text.split() if word.lower() not in stop_words])
    )

# Apply on the 'review' column
data['review'] = remove_stopwords_fast(data['review'])


In [38]:
data.tail(2)

,review,sentiment
49998,i'm going disagree previous comment side malti...,negative
49999,"one expects star trek movies high art, fans ex...",negative


In [39]:
#handling Emojis :

def remove_emoji(text):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"  # other symbols
        "\U000024C2-\U0001F251"  # enclosed characters
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", text)

In [40]:
data.review = data.review.apply(remove_emoji)

In [44]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [46]:
import nltk
print(nltk.data.path)


['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


In [54]:
import nltk
nltk.download('punkt', download_dir='/path/to/nltk_data')


# nltk baad me krenge

[nltk_data] Downloading package punkt to /path/to/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [53]:
import re
import spacy

# Define Tokenization Functions
def regex_tokenize(text):
    return re.split(r'\s+', text)



def spacy_tokenize(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    return [token.text for token in doc]

# Example Usage
text = "Hello world! This is an example sentence. Tokenization is fun 😊."

# Regular Expression Tokenization
print("Regex Tokenization:", regex_tokenize(text))



# SpaCy Tokenization
print("SpaCy Tokenization:", spacy_tokenize(text))


Regex Tokenization: ['Hello', 'world!', 'This', 'is', 'an', 'example', 'sentence.', 'Tokenization', 'is', 'fun', '😊.']
SpaCy Tokenization: ['Hello', 'world', '!', 'This', 'is', 'an', 'example', 'sentence', '.', 'Tokenization', 'is', 'fun', '😊', '.']


In [60]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [67]:
# Stemming :

"""from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')  # Ensure punkt tokenizer is downloaded

ps = PorterStemmer()

def stemming(text):
    words = word_tokenize(text)  # Tokenize into words
    stemmed_words = [ps.stem(word) for word in words]  # Stem each word
    return " ".join(stemmed_words)  # Join back into a string



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [69]:
#data.review = data.review.apply(stemming)

In [70]:
# lemmitation :

'''def lemmatization(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    lemmatized_words = [token.lemma_ for token in doc]
    return " ".join(lemmatized_words)

In [72]:
#data.review = data.review.apply(lemmatization)

**Corpus and all : **

In [73]:
import re

# Function to count total words and unique words
def count_words_and_vocabulary(text):
    # Tokenize the text (split by space or any non-alphabetic characters)
    words = re.findall(r'\b\w+\b', text.lower())  # Tokenize and convert to lowercase
    total_words = len(words)  # Count total words
    unique_words = len(set(words))  # Count unique words (vocabulary)
    return total_words, unique_words

# Apply the function to the 'review' column
total_words, unique_words = zip(*data['review'].apply(count_words_and_vocabulary))

# Sum up the total words and unique words across all reviews
total_words_count = sum(total_words)
unique_words_count = sum(unique_words)

print(f"Total words in the corpus: {total_words_count}")
print(f"Total unique words (vocabulary) in the corpus: {unique_words_count}")


Total words in the corpus: 6473541
Total unique words (vocabulary) in the corpus: 5266128


In [75]:
# One Hot Encoding :

from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd

# Assuming the 'review' column contains text data
# First we tokenize the text and then apply One Hot Encoding

# Tokenize the text data
def tokenize(text):
    return text.split()  # simple split by space

# Example dataset
data = pd.DataFrame({
    'review': ["I love programming", "Python is great", "AI is fascinating"]
})

# Apply the tokenization function to the 'review' column
data['tokens'] = data['review'].apply(tokenize)

# Flatten the list of tokens from all rows to get unique words for encoding
all_words = [word for tokens in data['tokens'] for word in tokens]
unique_words = list(set(all_words))

# One Hot Encoding using sklearn's OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
encoded_words = encoder.fit_transform(np.array(unique_words).reshape(-1, 1))

# Create a DataFrame for the One Hot Encoded data
one_hot_df = pd.DataFrame(encoded_words, columns=unique_words)

# Display the one-hot encoded DataFrame
print(one_hot_df)



     I  love   is  great   AI  Python  programming  fascinating
0  0.0   1.0  0.0    0.0  0.0     0.0          0.0          0.0
1  0.0   0.0  0.0    0.0  0.0     0.0          1.0          0.0
2  0.0   0.0  0.0    0.0  0.0     1.0          0.0          0.0
3  0.0   0.0  0.0    0.0  1.0     0.0          0.0          0.0
4  1.0   0.0  0.0    0.0  0.0     0.0          0.0          0.0
5  0.0   0.0  1.0    0.0  0.0     0.0          0.0          0.0
6  0.0   0.0  0.0    0.0  0.0     0.0          0.0          1.0
7  0.0   0.0  0.0    1.0  0.0     0.0          0.0          0.0


In [76]:
# applying BOW and finding vocalublury

from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer (it will automatically handle tokenization and counting)
vectorizer = CountVectorizer()

# Apply the vectorizer to your text data (e.g., review column)
X = vectorizer.fit_transform(data['review'])

# Convert the result to a DataFrame
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Vocabulary (unique words)
vocabulary = vectorizer.get_feature_names_out()

# Word occurrences (count of times each word appeared)
word_occurrences = bow_df.sum()

# Print the vocabulary and word occurrences
print("Vocabulary:", vocabulary)
print("Word Occurrences:", word_occurrences)


Vocabulary: ['ai' 'fascinating' 'great' 'is' 'love' 'programming' 'python']
Word Occurrences: ai             1
fascinating    1
great          1
is             2
love           1
programming    1
python         1
dtype: int64


In [77]:
# bi-gram , tri-gram :

from sklearn.feature_extraction.text import CountVectorizer

# Bag of Bi-grams
bi_vectorizer = CountVectorizer(ngram_range=(2, 2))
X_bi = bi_vectorizer.fit_transform(data['review'])
bi_vocab = bi_vectorizer.get_feature_names_out()
bi_bow_df = pd.DataFrame(X_bi.toarray(), columns=bi_vocab)
bi_word_occurrences = bi_bow_df.sum()

# Bag of Tri-grams
tri_vectorizer = CountVectorizer(ngram_range=(3, 3))
X_tri = tri_vectorizer.fit_transform(data['review'])
tri_vocab = tri_vectorizer.get_feature_names_out()
tri_bow_df = pd.DataFrame(X_tri.toarray(), columns=tri_vocab)
tri_word_occurrences = tri_bow_df.sum()

# Print Vocabulary and Word Occurrences for Bi-grams and Tri-grams
print("Bag of Bi-grams Vocabulary:", bi_vocab)
print("Bag of Bi-grams Word Occurrences:", bi_word_occurrences)

print("\nBag of Tri-grams Vocabulary:", tri_vocab)
print("Bag of Tri-grams Word Occurrences:", tri_word_occurrences)

# Dimensionality (number of features)
print("\nDimensionality of Bi-grams Vocabulary:", len(bi_vocab))
print("Dimensionality of Tri-grams Vocabulary:", len(tri_vocab))


Bag of Bi-grams Vocabulary: ['ai is' 'is fascinating' 'is great' 'love programming' 'python is']
Bag of Bi-grams Word Occurrences: ai is               1
is fascinating      1
is great            1
love programming    1
python is           1
dtype: int64

Bag of Tri-grams Vocabulary: ['ai is fascinating' 'python is great']
Bag of Tri-grams Word Occurrences: ai is fascinating    1
python is great      1
dtype: int64

Dimensionality of Bi-grams Vocabulary: 5
Dimensionality of Tri-grams Vocabulary: 2


In [79]:
# Apply tf-idf and find out the idf scores of words, also find out the vocabulary.


from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit the model on the data and transform it to get TF-IDF scores
X_tfidf = tfidf_vectorizer.fit_transform(data['review'])

# Get IDF scores of words
idf_scores = tfidf_vectorizer.idf_

# Get the vocabulary (words)
vocabulary = tfidf_vectorizer.get_feature_names_out()

# Create a DataFrame for better readability
idf_df = pd.DataFrame(idf_scores, index=vocabulary, columns=['IDF Score'])

# Print vocabulary and IDF scores
print("Vocabulary:", vocabulary)
print("\nIDF Scores:\n", idf_df)

# You can also check the shape of the TF-IDF matrix (dimensionality)
print("\nDimensionality of TF-IDF Matrix:", X_tfidf.shape)


Vocabulary: ['ai' 'fascinating' 'great' 'is' 'love' 'programming' 'python']

IDF Scores:
              IDF Score
ai            1.693147
fascinating   1.693147
great         1.693147
is            1.287682
love          1.693147
programming   1.693147
python        1.693147

Dimensionality of TF-IDF Matrix: (3, 7)
